In [2]:
import torch
from torch.autograd import Function
from torch.autograd import gradcheck


In [3]:
class Context:
    """Un objet contexte très simplifié pour simuler PyTorch

    Un contexte différent doit être utilisé à chaque forward
    """
    def __init__(self):
        self._saved_tensors = ()
    def save_for_backward(self, *args):
        self._saved_tensors = args
    @property
    def saved_tensors(self):
        return self._saved_tensors


class MSE(Function):
    """Début d'implementation de la fonction MSE"""
    @staticmethod
    def forward(ctx, yhat, y):
        ## Garde les valeurs nécessaires pour le backwards
        ctx.save_for_backward(yhat, y)

        return (y-yhat).pow(2).sum().mean()

    @staticmethod
    def backward(ctx, grad_output):
        ## Calcul du gradient du module par rapport a chaque groupe d'entrées
        yhat, y = ctx.saved_tensors
        #  TODO:  Renvoyer par les deux dérivées partielles (par rapport à yhat et à y)
        return grad_output*-2*(y-yhat), grad_output*2*(y-yhat)

#  TODO:  Implémenter la fonction Linear(X, W, b)sur le même modèle que MSE
class Linear(Function):
    @staticmethod
    def forward(ctx, X, W, b):
        ctx.save_for_backward(X, W, b)
        return X@W+b
    
    @staticmethod
    def backward(ctx, grad_output):
        X, W, b= ctx.saved_tensors
        return grad_output@W.T, X.T@grad_output, grad_output

    
## Utile dans ce TP que pour le script tp1_gradcheck
mse = MSE.apply
linear = Linear.apply

In [54]:
yhat = torch.randn(10,5, requires_grad=True, dtype=torch.float64)
y = torch.randn(10,5, requires_grad=True, dtype=torch.float64)
torch.autograd.gradcheck(mse, (yhat, y))

True

In [55]:
x = torch.randn(3, 2, requires_grad=True, dtype = torch.float64)
w = torch.randn(2, 4, requires_grad=True, dtype = torch.float64)
b = torch.randn(4, requires_grad=True, dtype = torch.float64)

# torch.autograd.gradcheck(linear, (x,w, b))
y = torch.randn(3, 4)
yhat = linear(x,w,b)
assert yhat.shape == y.shape
loss = mse(yhat, y)
loss.backward()
w.grad

tensor([[ 6.2917, -1.6823, -0.1485,  4.4168],
        [-6.5817, -3.6313,  6.7623,  1.0391]], dtype=torch.float64)

In [4]:
import torch
from torch.utils.tensorboard import SummaryWriter
from tp1 import MSE, Linear, Context

# Les données supervisées
x = torch.randn(50, 13, requires_grad=True)
y = torch.randn(50, 3)

# Les paramètres du modèle à optimiser
w = torch.randn(13, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)

epsilon = 0.001

writer = SummaryWriter()
for n_iter in range(200):
    ##  TODO:  Calcul du forward (loss)
    yhat = linear(x,w,b)
    loss = mse(yhat, y)
    # `loss` doit correspondre au coût MSE calculé à cette itération
    # on peut visualiser avec
    # tensorboard --logdir runs/
    writer.add_scalar('Loss/train', loss, n_iter)

    # Sortie directe
    print(f"Itérations {n_iter}: loss {loss}")

    # Calcul du backward (grad_w, grad_b)
    loss.backward()
    # Mise à jour des paramètres du modèle
    with torch.no_grad():
        w -= epsilon*w.grad
        b -= epsilon*b.grad
        w.grad = None
        b.grad = None

Itérations 0: loss 2585.802001953125
Itérations 1: loss 2002.4383544921875
Itérations 2: loss 1571.7857666015625
Itérations 3: loss 1251.0384521484375
Itérations 4: loss 1009.9608764648438
Itérations 5: loss 827.0586547851562
Itérations 6: loss 686.9537353515625
Itérations 7: loss 578.5724487304688
Itérations 8: loss 493.8895263671875
Itérations 9: loss 427.0503845214844
Itérations 10: loss 373.7559814453125
Itérations 11: loss 330.8287048339844
Itérations 12: loss 295.9032287597656
Itérations 13: loss 267.2072448730469
Itérations 14: loss 243.40325927734375
Itérations 15: loss 223.4746551513672
Itérations 16: loss 206.64321899414062
Itérations 17: loss 192.3087615966797
Itérations 18: loss 180.00485229492188
Itérations 19: loss 169.36639404296875
Itérations 20: loss 160.10531616210938
Itérations 21: loss 151.99261474609375
Itérations 22: loss 144.84490966796875
Itérations 23: loss 138.51412963867188
Itérations 24: loss 132.8798828125
Itérations 25: loss 127.84352111816406
Itérations 2